# Programming Exercise 3: Propositional Logic

## Initialization

In [4]:
from utils import expr
from kb import DpllPropKB, FCPropKB
from main_loop import main_loop
from minefield import Minefield
import maps

## Knowledge Base generation
### Variables are:

$B_{i, j}$ there is a bomb in `[i, j]`.<br/>
$O_{i, j}$ There is one bomb near `[i, j]`.<br/>
$TW_{i, j}$ There are two bombs near `[i, j]`.<br/>
$TH_{i, j}$ There are three bombs near `[i, j]`.<br/>
$F_{i, j}$ There are four bombs near `[i, j]`.<br/>
$Z_{i, j}$ There is no bomb near `[i, j]`.<br/>

### Auxiliary functions

The following auxiliary functions will help you generate propositions with those variables:

In [5]:
def field_var(sym, x, y):
    assert type(x) == int and type(y) == int
    return "%s%i%i" % (sym, x, y)

def v_bomb(x, y):
    return field_var("B", x, y)

def v_zero(x, y):
    return field_var("Z", x, y)

def v_one(x, y):
    return field_var("O", x, y)

def v_two(x, y):
    return field_var("TW", x, y)

def v_three(x, y):
    return field_var("TH", x, y)

def v_four(x, y):
    return field_var("F", x, y)

### Sample knowledge base generation

This in an example that demonstrates how propositional sentences can be added to the knowledge base.

In [6]:
def generate_knowledge_example(dim_x = 7, dim_y = 7):
    kb = []
    for x in range(dim_x):
        for y in range(dim_y):
            new_proposition = v_bomb(x, y) + " | ~" + v_bomb(x, y) # Bxy ∨ ¬Bxy
            kb.append(new_proposition)
            
            new_proposition = v_zero(x, y) + " ==> " + v_zero(x, y) # Zxy ==> Zxy
            kb.append(new_proposition)
            
            new_proposition = v_two(x, y) + " & ~" + v_two(x, y) + " ==> False" # TWxy ∧ ¬TWxy ==> False
            kb.append(new_proposition)
    
    return kb

## Your Knowledge Base
* There is no bomb in field `[0,0]`
* There is no bomb in field `[6,6]`
* The neighboring for field `[i,j]` are `[i-1,j]`, `[i, j+1]`, `[i+1, j]`, `[i, j-1]` (in border and corner cases, there are fewer)

In [63]:
def generate_knowledge(dim_x = 7, dim_y = 7):
    # TODO generate the Knowledge Base
    # you can assume dim_x and dim_y are always 7
    kb = []
    for x in range(dim_x):
        for y in range(dim_y):
            if x==0 and y==0:
                new_proposition = "~" + v_bomb(x,y) + " & " + v_zero(x,y) + " ==> ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x,y+1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_one(x,y) + " ==> " + v_bomb(x+1,y) + " & ~" + v_bomb(x,y+1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x,y+1) 
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_two(x,y) + " ==> " + v_bomb(x+1,y) + " & " + v_bomb(x,y+1)
                kb.append(new_proposition)
                continue
            
            if x==0 and y==6:
                new_proposition = "~" + v_bomb(x,y) + " & " + v_zero(x,y) + " ==> ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x,y-1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_one(x,y) + " ==> " + v_bomb(x+1,y) + " & ~" + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x,y-1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_two(x,y) + " ==> " + v_bomb(x+1,y) + " & " + v_bomb(x,y-1)
                kb.append(new_proposition)
                continue
                
            if x==6 and y==0:
                new_proposition = "~" + v_bomb(x,y) + " & " + v_zero(x,y) + " ==> ~" + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_one(x,y) + " ==> " + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " | ~" + v_bomb(x-1,y) + " & " + v_bomb(x,y+1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_two(x,y) + " ==> " + v_bomb(x-1,y) + " & " + v_bomb(x,y+1)
                kb.append(new_proposition)
                continue
                
            if x==0 and y!=0 and y!=6:
                new_proposition = "~" + v_bomb(x,y) + " & " + v_zero(x,y) + " ==> ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_one(x,y) + " ==> " + v_bomb(x+1,y) + " & ~" + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x,y+1) + " & " + v_bomb(x,y-1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_two(x,y) + " ==> " + v_bomb(x+1,y) + " & " + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | " + v_bomb(x+1,y) + " & ~" + v_bomb(x,y+1) + " & " + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x,y+1) + " & " + v_bomb(x,y-1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_three(x,y) + " ==> " + v_bomb(x+1,y) + " & " + v_bomb(x,y+1) + " & " + v_bomb(x,y-1)
                kb.append(new_proposition)
                continue
                
            if x==6 and y!=0 and y!=6:
                new_proposition = "~" + v_bomb(x,y) + " & " + v_zero(x,y) + " ==> ~" + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_one(x,y) + " ==> " + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | ~" + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | ~" + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & " + v_bomb(x,y-1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_two(x,y) + " ==> " + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | " + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & " + v_bomb(x,y-1) + " | ~" + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & " + v_bomb(x,y-1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_three(x,y) + " ==> " + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & " + v_bomb(x,y-1)
                kb.append(new_proposition)
                continue
                
            if x!=0 and x!=6 and y==0:
                new_proposition = "~" + v_bomb(x,y) + " & " + v_zero(x,y) + " ==> ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_one(x,y) + " ==> " + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " | ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & " + v_bomb(x,y+1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_two(x,y) + " ==> " + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " | " + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & " + v_bomb(x,y+1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_three(x,y) + " ==> " + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & " + v_bomb(x,y+1)
                kb.append(new_proposition)
                continue
                
            if x!=0 and x!=6 and y==6:
                new_proposition = "~" + v_bomb(x,y) + " & " + v_zero(x,y) + " ==> ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & ~" + v_bomb(x,y-1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_one(x,y) + " ==> " + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & ~" + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & ~" + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & " + v_bomb(x,y-1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_two(x,y) + " ==> " + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & ~" + v_bomb(x,y-1) + " | " + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & " + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & " + v_bomb(x,y-1)
                kb.append(new_proposition)
                
                new_proposition = "~" + v_bomb(x,y) + " & " + v_three(x,y) + " ==> " + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & " + v_bomb(x,y-1)
                kb.append(new_proposition)
                continue
                
            
            new_proposition = "~" + v_bomb(x,y) + " & " + v_zero(x,y) + " ==> ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1)
            kb.append(new_proposition)
            
            new_proposition = "~" + v_bomb(x,y) + " & " + v_one(x,y) + " ==> " + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & " + v_bomb(x,y-1)
            kb.append(new_proposition)
            
            new_proposition = "~" + v_bomb(x,y) + " & " + v_two(x,y) + " ==> " + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | " + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | " + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & " + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & " + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & " + v_bomb(x,y-1)
            kb.append(new_proposition)

            new_proposition = "~" + v_bomb(x,y) + " & " + v_three(x,y) + " ==> " + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & ~" + v_bomb(x,y-1) + " | " + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & ~" + v_bomb(x,y+1) + " & " + v_bomb(x,y-1) + " | " + v_bomb(x+1,y) + " & ~" + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & " + v_bomb(x,y-1) + " | ~" + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & " + v_bomb(x,y-1)
            kb.append(new_proposition)
            
            new_proposition = "~" + v_bomb(x,y) + " & " + v_four(x,y) + " ==> " + v_bomb(x+1,y) + " & " + v_bomb(x-1,y) + " & " + v_bomb(x,y+1) + " & " + v_bomb(x,y-1)
            kb.append(new_proposition)
            
            pass
        
        
    new_proposition = "~" + v_bomb(0,0) + " & ~" + v_bomb(6,6)
    kb.append(new_proposition)
    
    return kb

## Configuration
### Choose a minefield:
(A random field could be unsolvable)

In [91]:
# possible values: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, "random"
minefield_gen = 1

if minefield_gen == "random":
    minefield = Minefield(dim_x = 7, dim_y = 7, mine_count=10)
else:
    minefield = Minefield(mine_map = getattr(maps, "mine_map"+str(minefield_gen)))

### Choose a knowledge base class:

#### possible values:
* "Dpll"
    - use this knowledge base for dpll proving
    - this works for every kind of knowledge base
* "FC"
    - use this knowledge base for proving with forward chaining
    - warning: the knowledge base should only contain clauses in the following forms for this to work:
        * α & .. & β ==> γ & .. & δ
        * α & .. & β <== γ & .. & δ
        * α & .. & β <=> γ & .. & δ
        * α & .. & β

In [92]:
# possible values: "Dpll", "FC"
kb_gen = "Dpll"

KB = globals()[kb_gen+"PropKB"]

## Simulation
### The Robot has to find a way through the minefield:
* Valid actions are 'up','down','left' and 'right'
* The robot get's information of the field he is on. For example: $¬B_{i, j} ∧ Z_{i, j}$
* Walking on a mine kills the robot

In [93]:
kb = KB() # create empty knowledge base

print("feed knowledge base with knowledge..")
for str_expr in generate_knowledge(minefield.dim_x, minefield.dim_y):
    kb.tell(expr(str_expr))

# check if the knowledge base is obviosly wrong (you can remove this if it is too slow)
print("scan knowledge base for contradictions..")
assert not kb.has_contradicting_knowledge()

# run the simulation
# when you set stop_on_risk to False, the simulation will continue with guessing when necessary
# to speed up the simulation, set loop_delay to 0
main_loop(minefield, kb, stop_on_risk = True, loop_delay = 0)

feed knowledge base with knowledge..
scan knowledge base for contradictions..


ask: ~B01 ≡ True
target field: (0, 1) 


ask: ~B11 ≡ True
target field: (1, 1) 


ask: ~B12 ≡ True
target field: (1, 2) 


ask: ~B13 ≡ True
target field: (1, 3) 


...

ask: ~B56 ≡ ?
ask: ~B45 ≡ True
target field: (4, 5) 


ask: ~B55 ≡ True
target field: (5, 5) 


ask: ~B56 ≡ ?
ask: ~B65 ≡ True
target field: (6, 5) 


ask: ~B66 ≡ True
target field: (6, 6) 


Victory!!!
